## Creación de data temporal

### Importación de datos de archivos `CSV`

In [10]:
import pandas as pd
import os
import re

columnas = ['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 
            'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 
            'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

carpeta = 'data/csv/'
dfs = []

for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):
        ruta_completa = os.path.join(carpeta, archivo)
        temp_df = pd.read_csv(ruta_completa)
        print(f'Leyendo {archivo} con {temp_df.shape[0]} centros educativos')
        dfs.append(pd.read_csv(ruta_completa))

df = pd.concat(dfs, ignore_index=True)

Leyendo altaverapaz.csv con 294 centros educativos
Leyendo bajaverapaz.csv con 94 centros educativos
Leyendo chimaltenango.csv con 300 centros educativos
Leyendo chiquimula.csv con 136 centros educativos
Leyendo ciudadcapital.csv con 860 centros educativos
Leyendo elprogreso.csv con 97 centros educativos
Leyendo escuintla.csv con 393 centros educativos
Leyendo guatemala.csv con 1036 centros educativos
Leyendo huehuetenango.csv con 295 centros educativos
Leyendo izabal.csv con 273 centros educativos
Leyendo jalapa.csv con 121 centros educativos
Leyendo jutiapa.csv con 296 centros educativos
Leyendo peten.csv con 270 centros educativos
Leyendo quetzaltenango.csv con 365 centros educativos
Leyendo quiche.csv con 184 centros educativos
Leyendo retalhuleu.csv con 272 centros educativos
Leyendo sacatepequez.csv con 206 centros educativos
Leyendo sanmarcos.csv con 431 centros educativos
Leyendo santarosa.csv con 133 centros educativos
Leyendo solola.csv con 111 centros educativos
Leyendo such

### Preparación de carpeta destino

In [11]:
# datos limpios
import os
import shutil

carpeta_destino = 'data_temporal1'
carpeta_destino_csv = os.path.join(carpeta_destino, 'csv')

# Crear la carpeta si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
    print(f"Carpeta '{carpeta_destino}' creada.")

if not os.path.exists(carpeta_destino_csv):
    os.makedirs(carpeta_destino_csv)
    print(f"Carpeta '{carpeta_destino_csv}' creada.")

In [12]:
# 1. Misma dirección pero diferentes nombres
direcciones_counts = df.groupby('DIRECCION').size()
direcciones_duplicadas = direcciones_counts[direcciones_counts > 1].sort_values(ascending=False)

print(f"Existen {len(direcciones_duplicadas)} direcciones que aparecen en más de un registro.")
print(f"Las 5 direcciones más repetidas son:")
for direccion in direcciones_duplicadas.head(5).index:
    print(f"\nDirección: '{direccion}'")
    print(df[df['DIRECCION'] == direccion][['CODIGO', 'ESTABLECIMIENTO', 'JORNADA', 'PLAN']].head(3))

# 2. Agrupamiento establecimientos similares
def simplificar_nombre(nombre):
    # Convertir a mayúsculas y eliminar comillas
    nombre = nombre.upper().replace('"', '')
    # Eliminar palabras genéricas que pueden causar variaciones
    palabras_eliminar = ['PRIVADO', 'MIXTO', 'COLEGIO', 'INSTITUTO', 'ESCUELA', 'CENTRO', 'DE', 'Y', 'EDUCACION']
    for palabra in palabras_eliminar:
        nombre = nombre.replace(f" {palabra} ", " ")
    # Eliminar espacios múltiples
    return " ".join(nombre.split())

# columna temporal con nombres simplificados
df['NOMBRE_SIMPLIFICADO'] = df['ESTABLECIMIENTO'].apply(simplificar_nombre)

# ocurrencias de nombres simplificados
nombres_simples_counts = df['NOMBRE_SIMPLIFICADO'].value_counts()
nombres_simples_duplicados = nombres_simples_counts[nombres_simples_counts > 1].sort_values(ascending=False)

print(f"\nDespués de simplificar nombres, existen {len(nombres_simples_duplicados)} nombres potencialmente similares.")
print(f"Los 5 nombres más comunes después de simplificación son:")
for nombre in nombres_simples_duplicados.head(5).index:
    print(f"\nNombre simplificado: '{nombre}'")
    print(f"Nombres originales:")
    print(df[df['NOMBRE_SIMPLIFICADO'] == nombre]['ESTABLECIMIENTO'].sample(min(3, nombres_simples_counts[nombre])).tolist())

# 3. Integridad de relaciones entre DEPARTAMENTO, MUNICIPIO y DISTRITO
print("\nVerificando integridad de relaciones geográficas:")
municipios_por_depto = df.groupby(['DEPARTAMENTO', 'MUNICIPIO']).size().reset_index()
print(f"Hay {len(municipios_por_depto)} combinaciones únicas de DEPARTAMENTO-MUNICIPIO")

# relación entre distrito y departamento
distrito_depto = df[['DISTRITO', 'DEPARTAMENTO']].drop_duplicates()
print(f"Hay {len(distrito_depto)} combinaciones únicas de DISTRITO-DEPARTAMENTO")

# distritos que pertenecen a más de un departamento (posible inconsistencia)
distrito_counts = distrito_depto.groupby('DISTRITO').size()
distritos_multiples = distrito_counts[distrito_counts > 1]
if len(distritos_multiples) > 0:
    print(f"Hay {len(distritos_multiples)} distritos asignados a más de un departamento (posible inconsistencia)")
    for distrito in distritos_multiples.index:
        print(f"\nDistrito: {distrito}")
        print(df[df['DISTRITO'] == distrito][['DEPARTAMENTO']].drop_duplicates().values.flatten())

Existen 1039 direcciones que aparecen en más de un registro.
Las 5 direcciones más repetidas son:

Dirección: 'CABECERA MUNICIPAL'
             CODIGO                                ESTABLECIMIENTO  \
361   15-05-0014-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   
367   15-06-0006-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   
1737  02-04-0004-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   

         JORNADA             PLAN  
361   VESPERTINA  DIARIO(REGULAR)  
367   VESPERTINA  DIARIO(REGULAR)  
1737  VESPERTINA  DIARIO(REGULAR)  

Dirección: 'BARRIO EL CENTRO'
            CODIGO                                ESTABLECIMIENTO     JORNADA  \
133  16-06-0090-46  INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA  VESPERTINA   
162  16-08-0208-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA  VESPERTINA   
241  16-13-0254-46  INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA    MATUTINA   

                PLAN  
133  DIARIO(REGULAR)  
162  DIARIO(REGULAR)  
241  DIARIO(REGULAR

In [13]:
# Implementar las transformaciones de limpieza
# Crear una copia del DataFrame original para no modificarlo
df_limpio = df.copy()

print("Aplicando transformaciones de limpieza...")

# 1. Limpieza de ESTABLECIMIENTO
print("Limpiando columna ESTABLECIMIENTO...")

# normalizar acentos en palabras comunes
def normalizar_acentos(texto):
    reemplazos = {
        'EDUCACION': 'EDUCACIÓN',
        'BASICO': 'BÁSICO',
        'TECNICO': 'TÉCNICO',
        'INFORMATICA': 'INFORMÁTICA',
        'TECNOLOGICO': 'TECNOLÓGICO',
        'CIENTIFICO': 'CIENTÍFICO',
        'MECANICA': 'MECÁNICA',
        'ELECTRONICA': 'ELECTRÓNICA',
        'GRAFICA': 'GRÁFICA',
        'DISENIO': 'DISEÑO',
        'COMERCIO': 'COMERCIO',
        'ADMINISTRACION': 'ADMINISTRACIÓN',
        'MUSICA': 'MÚSICA',
        'ECONOMIA': 'ECONOMÍA',
        'PEDAGOGICO': 'PEDAGÓGICO',
        'TECNOLOGIA': 'TECNOLOGÍA',
        'ELECTROMECANICA': 'ELECTROMECÁNICA',
        'AGRICOLA': 'AGRÍCOLA',
        'MECANICO': 'MECÁNICO',
        'QUIMICA': 'QUÍMICA',
        'ELECTRONICO': 'ELECTRÓNICO'
    }
    
    for palabra, reemplazo in reemplazos.items():
        texto = re.sub(r'\b' + palabra + r'\b', reemplazo, texto)
    
    return texto

# aplicar normalización de acentos
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].apply(normalizar_acentos)

# eliminación de  comillas y caracteres especiales innecesarios
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace('"', '')
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace('\'', '')

#  eliminación de espacios duplicados
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace(r'\s+', ' ', regex=True).str.strip()

# 2. Limpieza de DIRECCION
print("Limpiando columna DIRECCION...")

# Completar valores nulos
df_limpio['DIRECCION'] = df_limpio['DIRECCION'].fillna('SIN DIRECCIÓN')

# Normalizar abreviaturas comunes
direccion_reemplazos = {
    r'\bAV\b': 'AVENIDA',
    r'\bC\b': 'CALLE',
    r'\bCALLE\s+(\d+)': r'CALLE \1',
    r'\bAVENIDA\s+(\d+)': r'AVENIDA \1',
    r'\bZONA\s+(\d+)': r'ZONA \1',
    r'\bZ\.\s*(\d+)': r'ZONA \1',
    r'\bZ\s+(\d+)': r'ZONA \1',
    r'\bLOT\b': 'LOTE',
    r'\bSECTOR\s+(\d+)': r'SECTOR \1',
    r'\bSECT\s+(\d+)': r'SECTOR \1',
    r'\bKM\b': 'KILÓMETRO',
    r'\bKM\.\s*(\d+)': r'KILÓMETRO \1',
}

for patron, reemplazo in direccion_reemplazos.items():
    df_limpio['DIRECCION'] = df_limpio['DIRECCION'].str.replace(patron, reemplazo, regex=True)

# 3. limpieza de TELEFONO
print("Limpiando columna TELEFONO...")

df_limpio['TELEFONO'] = df_limpio['TELEFONO'].astype(str)

# Función para estandarizar el formato de teléfono
def estandarizar_telefono(telefono):
    if pd.isna(telefono) or telefono == 'nan':
        return 'NO DISPONIBLE'
    
    # Eliminar caracteres no numéricos
    solo_numeros = re.sub(r'[^\d]', '', telefono)
    # Si después de limpieza queda vacío
    if not solo_numeros:
        return 'NO DISPONIBLE'
    
    # Si tiene 8 dígitos, es un formato válido
    if len(solo_numeros) == 8:
        return solo_numeros
    
    # Si tiene más o menos dígitos, puede ser un error
    if len(solo_numeros) < 8:
        return 'FORMATO INCORRECTO: ' + telefono
    
    # Si tiene más de 8 dígitos, tomar los últimos 8
    return solo_numeros[-8:]

df_limpio['TELEFONO'] = df_limpio['TELEFONO'].apply(estandarizar_telefono)

# 4. SUPERVISOR y DIRECTOR
print("Limpiando columnas SUPERVISOR y DIRECTOR...")

# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return 'SIN ASIGNAR'
    
    # Convertir a mayúsculas
    nombre = nombre.upper()
    
    # Eliminar espacios duplicados
    nombre = re.sub(r'\s+', ' ', nombre).strip()
    
    return nombre

df_limpio['SUPERVISOR'] = df_limpio['SUPERVISOR'].apply(normalizar_nombre)
df_limpio['DIRECTOR'] = df_limpio['DIRECTOR'].apply(normalizar_nombre)

# 5. Creación de  columnas derivadas
print("Creando columnas derivadas...")

# Extraer el tipo de establecimiento del nombre
def extraer_tipo_establecimiento(nombre):
    tipos = ['COLEGIO', 'INSTITUTO', 'ESCUELA', 'LICEO', 'CENTRO', 'ACADEMIA']
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return 'OTRO'

df_limpio['TIPO_ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].apply(extraer_tipo_establecimiento)

# Crear un identificador único de establecimiento físico (combinando nombre simplificado y dirección)
df_limpio['ID_ESTABLECIMIENTO_FISICO'] = df_limpio['NOMBRE_SIMPLIFICADO'] + '_' + df_limpio['DIRECCION'].str.replace(' ', '_')

# Agrupar departamentos por región
def asignar_region(departamento):
    regiones = {
        'NORTE': ['ALTA VERAPAZ', 'BAJA VERAPAZ', 'PETÉN'],
        'SUR': ['ESCUINTLA', 'SANTA ROSA', 'SUCHITEPÉQUEZ', 'RETALHULEU'],
        'ORIENTE': ['CHIQUIMULA', 'EL PROGRESO', 'IZABAL', 'JALAPA', 'JUTIAPA', 'ZACAPA'],
        'OCCIDENTE': ['HUEHUETENANGO', 'QUETZALTENANGO', 'QUICHÉ', 'SAN MARCOS', 'SOLOLÁ', 'TOTONICAPÁN'],
        'CENTRAL': ['CHIMALTENANGO', 'GUATEMALA', 'SACATEPÉQUEZ']
    }
    
    # Manejar subdivisiones de Guatemala
    if 'GUATEMALA' in departamento:
        return 'CENTRAL'
    
    # Manejar subdivisiones de Quiché
    if 'QUICHÉ' in departamento:
        return 'OCCIDENTE'
    
    for region, deptos in regiones.items():
        if departamento in deptos:
            return region
    
    return 'NO DEFINIDA'

df_limpio['REGION'] = df_limpio['DEPARTAMENTAL'].apply(asignar_region)

# Crear una clasificación combinada
df_limpio['CLASIFICACION'] = df_limpio['SECTOR'] + '_' + df_limpio['MODALIDAD'] + '_' + df_limpio['PLAN'].str.replace(' ', '_')

print(f"Limpieza completada. DataFrame resultante tiene {df_limpio.shape[0]} filas y {df_limpio.shape[1]} columnas.")

Aplicando transformaciones de limpieza...
Limpiando columna ESTABLECIMIENTO...
Limpiando columna DIRECCION...
Limpiando columna TELEFONO...
Limpiando columnas SUPERVISOR y DIRECTOR...
Creando columnas derivadas...
Limpieza completada. DataFrame resultante tiene 6584 filas y 22 columnas.


In [14]:
# Guardar los datos limpios en archivos CSV por departamento
print("Guardando datos limpios en archivos CSV...")
departamentos = df_limpio['DEPARTAMENTAL'].unique()

# archivo general con todos los datos
archivo_general = os.path.join(carpeta_destino, 'centros_educativos_completo.csv')
df_limpio.to_csv(archivo_general, index=False)
print(f"Archivo general guardado en: {archivo_general}")

# archivos individuales por cada uno de los departamentos
for departamento in departamentos:
    # archivo (sin caracteres especiales)
    nombre_archivo = departamento.lower().replace(' ', '_').replace('é', 'e')
    nombre_archivo = re.sub(r'[^\w\-_\.]', '', nombre_archivo)
    # Filtrar datos por departamento
    df_depto = df_limpio[df_limpio['DEPARTAMENTAL'] == departamento]

    ruta_archivo = os.path.join(carpeta_destino_csv, f"{nombre_archivo}.csv")
    # Guardar archivo
    df_depto.to_csv(ruta_archivo, index=False)
    
    print(f"Guardado archivo para {departamento} con {df_depto.shape[0]} registros: {ruta_archivo}")

print(f"\nProceso completado. {len(departamentos)} archivos CSV generados en la carpeta {carpeta_destino_csv}")

Guardando datos limpios en archivos CSV...
Archivo general guardado en: data_temporal1\centros_educativos_completo.csv
Guardado archivo para ALTA VERAPAZ con 294 registros: data_temporal1\csv\alta_verapaz.csv
Guardado archivo para BAJA VERAPAZ con 94 registros: data_temporal1\csv\baja_verapaz.csv
Guardado archivo para CHIMALTENANGO con 300 registros: data_temporal1\csv\chimaltenango.csv
Guardado archivo para CHIQUIMULA con 136 registros: data_temporal1\csv\chiquimula.csv
Guardado archivo para GUATEMALA NORTE con 536 registros: data_temporal1\csv\guatemala_norte.csv
Guardado archivo para GUATEMALA ORIENTE con 281 registros: data_temporal1\csv\guatemala_oriente.csv
Guardado archivo para GUATEMALA OCCIDENTE con 551 registros: data_temporal1\csv\guatemala_occidente.csv
Guardado archivo para GUATEMALA SUR con 528 registros: data_temporal1\csv\guatemala_sur.csv
Guardado archivo para EL PROGRESO con 97 registros: data_temporal1\csv\el_progreso.csv
Guardado archivo para ESCUINTLA con 393 regis